# 网络中的网络（NiN）

前几节介绍的LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。本节我们介绍网络中的网络（NiN）[1]。它提出了另外一个思路，即串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络。


## NiN块

我们知道，卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维。回忆在[“多输入通道和多输出通道”](channels.ipynb)一节里介绍的$1\times 1$卷积层。它可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用$1\times 1$卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。图5.7对比了NiN同AlexNet和VGG等网络在结构上的主要区别。

![左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部](../img/nin.svg)

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的$1\times 1$卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。

In [ ]:
import tensorflow as tf
print(tf.__version__)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

网络中的网络(Network in Network, NiN)架构的基本构建块

1. 函数定义：
- 函数名为 `nin_block`，用于创建一个 NiN 块
- 接收四个参数：
  - num_channels：输出通道数
  - kernel_size：卷积核大小
  - strides：步长
  - padding：填充方式

2. 架构组成：
每个 NiN 块包含三个连续的卷积层：

- 第一层是标准卷积层：
  - 可配置卷积核大小、步长和填充方式
  - 使用 ReLU 激活函数
  - 输出通道数由 num_channels 参数指定

- 第二层是 1×1 卷积层：
  - 使用 1×1 的卷积核
  - 保持与第一层相同的输出通道数
  - 使用 ReLU 激活函数
  - 功能类似于全连接层

- 第三层也是 1×1 卷积层：
  - 配置与第二层相同
  - 进一步增强网络的非线性表达能力

3. 创新特点：
- 使用 1×1 卷积替代传统的全连接层
- 保持了特征的空间信息
- 减少了参数数量
- 增强了模型的非线性表达能力

4. 实现方式：
- 使用 TensorFlow 的 Keras Sequential API 构建
- 采用模块化设计，便于在更大的网络中重复使用

## 每一层的作用

### 第一层：标准卷积层
```python
blk.add(Conv2D(num_channels, kernel_size,
               strides=strides, padding=padding,
               activation='relu'))
```
作用：
1. 特征提取：
   - 通过可自定义大小的卷积核提取空间特征
   - 可以检测边缘、纹理等底层特征

2. 参数灵活性：
   - kernel_size：可调整卷积核大小
   - strides：控制特征图缩放
   - padding：控制输出大小

3. 非线性变换：
   - 使用 ReLU 激活引入非线性
   - 提高模型表达能力

### 第二层：1×1 卷积层
```python
blk.add(Conv2D(num_channels, kernel_size=1,
               activation='relu'))
```
作用：
1. 跨通道信息整合：
   - 在不同通道间进行特征组合
   - 相当于每个像素位置的全连接层

2. 降维作用：
   - 可以减少参数数量
   - 控制模型复杂度

3. 增加非线性：
   - 通过 ReLU 引入额外的非线性变换
   - 增强特征的表达能力

### 第三层：1×1 卷积层
```python
blk.add(Conv2D(num_channels, kernel_size=1, 
               activation='relu'))
```
作用：
1. 深度特征提取：
   - 进一步提取更抽象的特征
   - 增强模型的表示能力

2. 模型正则化：
   - 多层结构帮助防止过拟合
   - 提高模型泛化能力

3. 特征重组合：
   - 对第二层的输出进行再次整合
   - 产生更高层次的特征表示

### 整体架构优势：

1. 参数效率：
   - 1×1 卷积大大减少了参数数量
   - 相比传统全连接层更加高效

2. 空间信息保持：
   - 保留了特征的空间结构
   - 有利于后续层的处理

3. 多尺度特征：
   - 通过多层架构捕获不同层次特征
   - 提高模型的表达能力

4. 计算效率：
   - 1×1 卷积计算量小
   - 适合在资源受限环境使用


## 计算每一层神经网络的权重参数数量


### NiN块第一层 - 标准卷积层
```python
Conv2D(num_channels, kernel_size, strides=strides, padding=padding, activation='relu')
```
参数计算公式:
```
参数数量 = (kernel_height × kernel_width × input_channels × output_channels) + output_channels
```
- kernel_height × kernel_width: 卷积核大小
- input_channels: 输入通道数  
- output_channels: 输出通道数(num_channels)
- +output_channels: 偏置项

### NiN块第二层 - 1×1卷积层  
```python
Conv2D(num_channels, kernel_size=1, activation='relu')
```
参数计算公式:
```
参数数量 = (1 × 1 × input_channels × output_channels) + output_channels
```
- 1×1: 卷积核大小为1×1
- input_channels: 等于第一层的输出通道数(num_channels)
- output_channels: 输出通道数(num_channels) 
- +output_channels: 偏置项

### NiN块第三层 - 1×1卷积层
```python
Conv2D(num_channels, kernel_size=1, activation='relu')  
```
参数计算与第二层相同:
```
参数数量 = (1 × 1 × input_channels × output_channels) + output_channels
```

### 完整示例计算
以第一个NiN块为例:
```python
net.add(nin_block(96, kernel_size=11, strides=4, padding='valid'))
```

第一层参数:
- 卷积核: 11×11
- 输入通道: 1(灰度图像)
- 输出通道: 96
```
参数数量 = (11 × 11 × 1 × 96) + 96 = 11,616
```

第二层参数:
- 卷积核: 1×1 
- 输入通道: 96
- 输出通道: 96
```
参数数量 = (1 × 1 × 96 × 96) + 96 = 9,312
```

第三层参数:
- 与第二层相同
```
参数数量 = (1 × 1 × 96 × 96) + 96 = 9,312
```

第一个NiN块总参数:
```
总参数 = 11,616 + 9,312 + 9,312 = 30,240
```

同理可以计算其他NiN块:

第二个NiN块(256输出通道):
```python
net.add(nin_block(256, kernel_size=5, strides=1, padding='same'))
```
- 第一层: (5 × 5 × 96 × 256) + 256 = 614,656
- 第二层: (1 × 1 × 256 × 256) + 256 = 65,792  
- 第三层: (1 × 1 × 256 × 256) + 256 = 65,792
- 总参数: 746,240

第三个NiN块(384输出通道):
```python
net.add(nin_block(384, kernel_size=3, strides=1, padding='same'))  
```
- 第一层: (3 × 3 × 256 × 384) + 384 = 885,120
- 第二层: (1 × 1 × 384 × 384) + 384 = 147,840
- 第三层: (1 × 1 × 384 × 384) + 384 = 147,840  
- 总参数: 1,180,800

通过这种计算,我们可以看到:
1. 参数量主要集中在第一个卷积层
2. 1×1卷积层大大减少了参数数量
3. 随着通道数增加,参数量也显著增加

这种设计在保持模型表达能力的同时,有效控制了参数数量,是一个比较高效的架构。


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D

def nin_block(num_channels, kernel_size, strides, padding):
    """
    创建一个网络中的网络(NiN)块。

    参数:
    num_channels (int): 卷积层的输出通道数
    kernel_size (int 或 tuple): 卷积核的大小
    strides (int 或 tuple): 卷积步长
    padding (str): 填充方式，'valid' 或 'same'

    返回:
    tf.keras.models.Sequential: NiN块的顺序模型
    """
    # 创建一个顺序模型作为NiN块
    blk = Sequential()
    
    # 添加第一个卷积层，可自定义参数
    blk.add(Conv2D(num_channels, kernel_size,
                   strides=strides, padding=padding,
                   activation='relu'))
    
    # 添加第二个1x1卷积层，相当于全连接层
    blk.add(Conv2D(num_channels, kernel_size=1,
                   activation='relu'))
    
    # 添加第三个1x1卷积层，相当于全连接层
    blk.add(Conv2D(num_channels, kernel_size=1, 
                   activation='relu'))
    
    return blk

## NiN模型

NiN是在AlexNet问世不久后提出的。它们的卷积层设定有类似之处。NiN使用卷积窗口形状分别为$11\times 11$、$5\times 5$和$3\times 3$的卷积层，相应的输出通道数也与AlexNet中的一致。每个NiN块后接一个步幅为2、窗口形状为$3\times 3$的最大池化层。

除使用NiN块以外，NiN还有一个设计与AlexNet显著不同：NiN去掉了AlexNet最后的3个全连接层，取而代之地，NiN使用了输出通道数等于标签类别数的NiN块，然后使用全局平均池化层对每个通道中所有元素求平均并直接用于分类。这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层。NiN的这个设计的好处是可以显著减小模型参数尺寸，从而缓解过拟合。然而，该设计有时会造成获得有效模型的训练时间的增加。

### 1. 第一阶段：初始特征提取
```python
# 第一个NiN块
net.add(nin_block(96, kernel_size=11, strides=4, padding='valid'))
# 最大池化层
net.add(MaxPool2D(pool_size=3, strides=2))
```
- 使用大尺寸卷积核(11×11)进行初始特征提取
- 输出96个通道的特征图
- 通过最大池化压缩特征图尺寸，保留重要特征

### 2. 第二阶段：中层特征提取
```python
# 第二个NiN块
net.add(nin_block(256, kernel_size=5, strides=1, padding='same'))
# 最大池化层
net.add(MaxPool2D(pool_size=3, strides=2))
```
- 使用中等尺寸卷积核(5×5)提取更复杂的特征
- 增加到256个通道，提升特征表达能力
- 再次使用池化层降维

### 3. 第三阶段：高层特征提取
```python
# 第三个NiN块
net.add(nin_block(384, kernel_size=3, strides=1, padding='same'))
# 最大池化层
net.add(MaxPool2D(pool_size=3, strides=2))
```
- 使用小尺寸卷积核(3×3)提取高级特征
- 进一步增加到384个通道
- 继续使用池化层降维

### 4. 防止过拟合
```python
# Dropout层
net.add(Dropout(0.5))
```
- 添加Dropout层随机丢弃50%的神经元
- 有效防止模型过拟合

### 5. 分类阶段
```python
# 最后一个NiN块
net.add(nin_block(10, kernel_size=3, strides=1, padding='same'))
# 全局平均池化层
net.add(GlobalAveragePooling2D())
# 展平层
net.add(Flatten())
```
- 最后的NiN块将通道数减少到类别数(10)
- 使用全局平均池化替代全连接层
- 最终将特征展平为一维向量用于分类

### 架构特点：
1. 渐进式特征提取：
   - 从大卷积核到小卷积核
   - 从少通道到多通道
   
2. 多层次降维：
   - 使用步长卷积和池化层逐步降维
   - 保留重要特征信息

3. 创新设计：
   - 使用NiN块替代传统卷积层
   - 用全局平均池化替代全连接层
   
4. 防过拟合措施：
   - 使用Dropout层
   - 采用全局平均池化减少参数

## 计算每一层的权重数量

### 1. 第一个NiN块
输入：224×224×1 (灰度图像)
```python
net.add(nin_block(96, kernel_size=11, strides=4, padding='valid'))
```
第一层卷积(11×11):
- 参数 = (11×11×1×96) + 96 = 11,616

第二层卷积(1×1):
- 参数 = (1×1×96×96) + 96 = 9,312

第三层卷积(1×1):  
- 参数 = (1×1×96×96) + 96 = 9,312

第一个NiN块总参数：30,240

### 2. 最大池化层
```python
net.add(MaxPool2D(pool_size=3, strides=2))
```
- 参数数量：0 (池化层没有可训练参数)

### 3. 第二个NiN块
输入通道从96变为256
```python
net.add(nin_block(256, kernel_size=5, strides=1, padding='same'))
```
第一层卷积(5×5):
- 参数 = (5×5×96×256) + 256 = 614,656

第二层卷积(1×1):
- 参数 = (1×1×256×256) + 256 = 65,792

第三层卷积(1×1):
- 参数 = (1×1×256×256) + 256 = 65,792

第二个NiN块总参数：746,240

### 4. 最大池化层
```python
net.add(MaxPool2D(pool_size=3, strides=2))
```
- 参数数量：0

### 5. 第三个NiN块
输入通道从256变为384
```python
net.add(nin_block(384, kernel_size=3, strides=1, padding='same'))
```
第一层卷积(3×3):
- 参数 = (3×3×256×384) + 384 = 885,120

第二层卷积(1×1):
- 参数 = (1×1×384×384) + 384 = 147,840

第三层卷积(1×1):
- 参数 = (1×1×384×384) + 384 = 147,840

第三个NiN块总参数：1,180,800

### 6. 最大池化层
```python
net.add(MaxPool2D(pool_size=3, strides=2))
```
- 参数数量：0

### 7. Dropout层
```python
net.add(Dropout(0.5))
```
- 参数数量：0

### 8. 第四个NiN块(分类层)
输入通道从384变为10
```python
net.add(nin_block(10, kernel_size=3, strides=1, padding='same'))
```
第一层卷积(3×3):
- 参数 = (3×3×384×10) + 10 = 34,570

第二层卷积(1×1):
- 参数 = (1×1×10×10) + 10 = 110

第三层卷积(1×1):
- 参数 = (1×1×10×10) + 10 = 110

第四个NiN块总参数：34,790

### 9. 全局平均池化层
```python
net.add(GlobalAveragePooling2D())
```
- 参数数量：0

### 10. 展平层
```python
net.add(Flatten())
```
- 参数数量：0

### 总结
总参数数量 = 30,240 + 746,240 + 1,180,800 + 34,790 = 1,992,070

主要观察：
1. 参数主要集中在中间的NiN块
2. 第三个NiN块参数最多，占比约59%
3. 后面的层参数反而减少，这得益于使用全局平均池化
4. 通过这种设计，相比传统CNN大大减少了参数数量

这种参数分布显示了NiN网络的高效性：在保持强大特征提取能力的同时，通过精心的架构设计控制了模型的复杂度。

## 平均池化和Tensorflow API介绍
### 1. 平均池化(Average Pooling)的基本概念

平均池化是一种降采样操作，它的工作原理是：
```
# 示例：2x2平均池化
输入特征图片段：
[[1, 2],
 [3, 4]]

输出：(1 + 2 + 3 + 4) / 4 = 2.5
```

主要特点：
1. 数据压缩：
   - 将一个区域的值平均化为一个值
   - 减少特征图的空间维度

2. 特征提取：
   - 保留区域的平均特征
   - 对噪声有平滑作用

3. 参数无关：
   - 不需要训练参数
   - 计算过程固定

### 2. GlobalAveragePooling2D详解

#### 2.1 API定义
```python
tf.keras.layers.GlobalAveragePooling2D(
    data_format=None,
    keepdims=False,
    **kwargs
)
```

#### 2.2 主要参数
- data_format：数据格式
  - 'channels_last'：(batch, height, width, channels)
  - 'channels_first'：(batch, channels, height, width)
  
- keepdims：是否保持输出的维度
  - False：压缩维度
  - True：保持维度，但值为1

#### 2.3 工作原理
```python
# 示例：
输入形状：(batch_size, height, width, channels)
输出形状：(batch_size, channels)

# 具体计算过程
对每个通道：
1. 计算该通道所有像素的平均值
2. 得到一个标量值
```

#### 2.4 代码示例
```python
import tensorflow as tf

# 创建一个简单的测试数据
# 形状：(1, 4, 4, 2) - 1个样本，4x4大小，2个通道
input_data = tf.random.normal([1, 4, 4, 2])

# 应用全局平均池化
global_pool = tf.keras.layers.GlobalAveragePooling2D()
output = global_pool(input_data)

print("输入形状:", input_data.shape)  # (1, 4, 4, 2)
print("输出形状:", output.shape)      # (1, 2)
```

### 3. GlobalAveragePooling2D的优势

1. 减少参数数量：
2. 防止过拟合：
   - 没有需要训练的参数
   - 降低了模型复杂度

3. 保持空间信息：
   - 考虑了整个特征图的信息
   - 提供了全局的特征表示

4. 空间不变性：
   - 对输入大小的变化更加鲁棒
   - 提高模型泛化能力

### 4. 实际应用示例

```python
# 在NiN网络中的应用
model = tf.keras.Sequential([
    # ... 前面的卷积层 ...
    tf.keras.layers.Conv2D(num_classes, 1, activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    # 直接得到类别预测
])

# 可视化中间结果
class VisualizeModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv = tf.keras.layers.Conv2D(64, 3, activation='relu')
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        
    def call(self, inputs):
        x = self.conv(inputs)
        print("卷积后形状:", x.shape)
        x = self.gap(x)
        print("全局平均池化后形状:", x.shape)
        return x
```

### 5. 性能考虑

1. 计算效率：
   - 简单的平均操作
   - 计算速度快

2. 内存效率：
   - 不存储参数
   - 内存占用小

3. 梯度传播：
   - 梯度计算简单
   - 有助于训练稳定性


In [3]:
from tensorflow.keras.layers import MaxPool2D, Dropout, GlobalAveragePooling2D, Flatten

net = Sequential()
# 第一个NiN块，96个输出通道，11x11卷积核，步长4
net.add(nin_block(96, kernel_size=11, strides=4, padding='valid'))
# 最大池化层，3x3窗口，步长2
net.add(MaxPool2D(pool_size=3, strides=2))
# 第二个NiN块，256个输出通道，5x5卷积核，步长1
net.add(nin_block(256, kernel_size=5, strides=1, padding='same'))
# 最大池化层，3x3窗口，步长2
net.add(MaxPool2D(pool_size=3, strides=2))
# 第三个NiN块，384个输出通道，3x3卷积核，步长1
net.add(nin_block(384, kernel_size=3, strides=1, padding='same'))
# 最大池化层，3x3窗口，步长2
net.add(MaxPool2D(pool_size=3, strides=2))
# Dropout层，防止过拟合
net.add(Dropout(0.5))
# 最后一个NiN块，10个输出通道（对应10个类别），3x3卷积核，步长1
net.add(nin_block(10, kernel_size=3, strides=1, padding='same'))
# 全局平均池化层，将每个通道的特征图平均成一个值
net.add(GlobalAveragePooling2D())
# 展平层，将结果转换为一维向量
net.add(Flatten())

下面构造一个高和宽均为224的单通道数据样本来观察每一层的输出形状。

In [ ]:
X = tf.random.uniform((1,224,224,1))
for blk in net.layers:
    X = blk(X)
    print(blk.name, 'output shape:\t', X.shape)

## 获取数据和训练模型

我们依然使用Fashion-MNIST数据集来训练模型。NiN的训练与AlexNet和VGG的类似，但这里使用的学习率更大。

In [ ]:
import numpy as np

class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0,axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0,axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]
        
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.train_images[index],224,224,)
        return resized_images.numpy(), self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.test_images[index],224,224,)
        return resized_images.numpy(), self.test_labels[index]

batch_size = 128
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)

In [ ]:
def train_nin():
    import os
    
    weights_file_path = "5.8_nin.weights.h5"
    if os.path.exists(weights_file_path):
        net.load_weights(weights_file_path)
    else:
        print(f"权重文件 {weights_file_path} 未找到，跳过加载权重")
    epoch = 5
    num_iter = dataLoader.num_train//batch_size
    for e in range(epoch):
        for n in range(num_iter):
            x_batch, y_batch = dataLoader.get_batch_train(batch_size)
            net.fit(x_batch, y_batch)
            if n%20 == 0:
                net.save_weights("5.8_nin.weights.h5")
                
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.06, momentum=0.3, nesterov=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-7)
net.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

x_batch, y_batch = dataLoader.get_batch_train(batch_size)
net.fit(x_batch, y_batch)
train_nin()

我们将训练好的参数读入，然后取测试数据计算测试准确率

In [ ]:
# net.load_weights("5.8_nin.weights.h5")

x_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(x_test, y_test, verbose=2)

## 可视化展示模型最后一层的特征值

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# 获取第一个 NiN 块的权重
first_nin_block = net.layers[0]
first_conv_layer = first_nin_block.layers[0]  # 获取 NiN 块中的第一个卷积层
first_layer_weights = first_conv_layer.get_weights()[0]

# 获取权重的形状
weight_shape = first_layer_weights.shape
print("第一层卷积核形状:", weight_shape)

# 创建一个函数来可视化权重
def visualize_first_layer(weights):
    # 假设权重形状为 (11, 11, 1, 96)，如果不是，可能需要调整
    n_filters = weights.shape[3]
    n_rows = int(np.ceil(np.sqrt(n_filters)))
    n_cols = int(np.ceil(n_filters / n_rows))
    
    plt.figure(figsize=(20, 20))
    for i in range(n_filters):
        ax = plt.subplot(n_rows, n_cols, i + 1)
        
        # 获取单个滤波器的权重
        w = weights[:, :, 0, i]
        
        # 标准化权重
        w_normalized = stats.zscore(w.flatten()).reshape(w.shape)
        
        plt.imshow(w_normalized, cmap='viridis')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f'Filter {i+1}')
    
    plt.tight_layout()
    plt.show()

# 可视化第一层卷积核
visualize_first_layer(first_layer_weights)

# 打印权重的统计信息
print("\n原始权重:")
print("  最小值:", np.min(first_layer_weights))
print("  最大值:", np.max(first_layer_weights))
print("  平均值:", np.mean(first_layer_weights))
print("  标准差:", np.std(first_layer_weights))

# 计算标准化后的权重统计信息
normalized_weights = stats.zscore(first_layer_weights.flatten()).reshape(first_layer_weights.shape)
print("\n标准化后的权重:")
print("  最小值:", np.min(normalized_weights))
print("  最大值:", np.max(normalized_weights))
print("  平均值:", np.mean(normalized_weights))
print("  标准差:", np.std(normalized_weights))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# 获取第二个 NiN 块的权重
second_nin_block = net.layers[2]  # 假设第二个 NiN 块是网络的第三层（索引为2）
second_conv_layer = second_nin_block.layers[0]  # 获取 NiN 块中的第一个卷积层
second_layer_weights = second_conv_layer.get_weights()[0]

# 获取权重的形状
weight_shape = second_layer_weights.shape
print("第二层卷积核形状:", weight_shape)

# 创建一个函数来可视化权重
def visualize_second_layer(weights, num_filters_to_show=64):
    n_filters = min(weights.shape[3], num_filters_to_show)
    n_channels = weights.shape[2]
    n_rows = int(np.ceil(np.sqrt(n_filters)))
    n_cols = int(np.ceil(n_filters / n_rows))
    
    plt.figure(figsize=(20, 20))
    for i in range(n_filters):
        ax = plt.subplot(n_rows, n_cols, i + 1)
        
        # 对所有输入通道的权重取平均
        w = np.mean(weights[:, :, :, i], axis=2)
        
        # 标准化权重
        w_normalized = stats.zscore(w.flatten()).reshape(w.shape)
        
        plt.imshow(w_normalized, cmap='viridis')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f'Filter {i+1}')
    
    plt.tight_layout()
    plt.show()

# 可视化第二层卷积核
visualize_second_layer(second_layer_weights)

# 打印权重的统计信息
print("\n原始权重:")
print("  最小值:", np.min(second_layer_weights))
print("  最大值:", np.max(second_layer_weights))
print("  平均值:", np.mean(second_layer_weights))
print("  标准差:", np.std(second_layer_weights))

# 计算标准化后的权重统计信息
normalized_weights = stats.zscore(second_layer_weights.flatten()).reshape(second_layer_weights.shape)
print("\n标准化后的权重:")
print("  最小值:", np.min(normalized_weights))
print("  最大值:", np.max(normalized_weights))
print("  平均值:", np.mean(normalized_weights))
print("  标准差:", np.std(normalized_weights))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# 获取第三个 NiN 块的权重
third_nin_block = net.layers[4]  # 假设第三个 NiN 块是网络的第五层（索引为4）
third_conv_layer = third_nin_block.layers[0]  # 获取 NiN 块中的第一个卷积层
third_layer_weights = third_conv_layer.get_weights()[0]

# 获取权重的形状
weight_shape = third_layer_weights.shape
print("第三层卷积核形状:", weight_shape)

# 创建一个函数来可视化权重
def visualize_third_layer(weights, num_filters_to_show=64):
    n_filters = min(weights.shape[3], num_filters_to_show)
    n_rows = int(np.ceil(np.sqrt(n_filters)))
    n_cols = int(np.ceil(n_filters / n_rows))
    
    plt.figure(figsize=(20, 20))
    for i in range(n_filters):
        ax = plt.subplot(n_rows, n_cols, i + 1)
        
        # 对所有输入通道的权重取平均
        w = np.mean(weights[:, :, :, i], axis=2)
        
        # 标准化权重
        w_normalized = stats.zscore(w.flatten()).reshape(w.shape)
        
        plt.imshow(w_normalized, cmap='viridis')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f'Filter {i+1}')
    
    plt.tight_layout()
    plt.show()

# 可视化第三层卷积核
visualize_third_layer(third_layer_weights)

# 打印权重的统计信息
print("\n原始权重:")
print("  最小值:", np.min(third_layer_weights))
print("  最大值:", np.max(third_layer_weights))
print("  平均值:", np.mean(third_layer_weights))
print("  标准差:", np.std(third_layer_weights))

# 计算标准化后的权重统计信息
normalized_weights = stats.zscore(third_layer_weights.flatten()).reshape(third_layer_weights.shape)
print("\n标准化后的权重:")
print("  最小值:", np.min(normalized_weights))
print("  最大值:", np.max(normalized_weights))
print("  平均值:", np.mean(normalized_weights))
print("  标准差:", np.std(normalized_weights))

## 小结

* NiN重复使用由卷积层和代替全连接层的$1\times 1$卷积层构成的NiN块来构建深层网络。
* NiN去除了容易造成过拟合的全连接输出层，而是将其替换成输出通道数等于标签类别数的NiN块和全局平均池化层。
* NiN的以上设计思想影响了后面一系列卷积神经网络的设计。